# Transfer Learning: EfficientNetB3

We will use a pre-trained model with EfficientNetB3 architecture with "imageNet" weights. 

### imports

In [1]:
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D, Flatten, AvgPool2D
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt


### Generators for loading and augmenting images

In [2]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      shear_range=0.1,
#                                      zoom_range=0.1,
#                                      rotation_range=5,
#                                      width_shift_range=0.1,
#                                      height_shift_range=0.1,
#                                      fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Model

In [3]:
class XrayModel(tf.keras.Model):
    def __init__(self):
        super(XrayModel, self).__init__()

        self.base_model = tf.keras.applications.EfficientNetB3(input_shape=(150, 150, 3),
                                                               include_top=False,
                                                               weights='imagenet')
        for layer in self.base_model.layers:
            layer.trainable = False
        for layer in self.base_model.layers[-10:]:
            layer.trainable = True

        self.pool = AvgPool2D()
        self.flatten = Flatten()
        self.classifier = Dense(3, activation='softmax')
    
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.pool(x)
        x = self.flatten(x)
        outputs = self.classifier(x)
        return outputs

model = XrayModel()


2022-03-29 11:57:04.853140: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


43950080/43941136 [==============================] - 6s 0us/step


## Load the saved model

In [3]:
model = tf.keras.models.load_model('../models/efficientNetB3')

2022-03-29 10:19:17.526229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Compile and train

In [4]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics='accuracy')


In [5]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.90:
            self.model.stop_training = True


In [6]:
model.fit(train_generator,
         steps_per_epoch=100,
         epochs=5,
         validation_data=test_generator,
         validation_steps=3,
         callbacks=[MyCallback(), 
                    EarlyStopping(monitor='val_loss', mode='min',
                                  verbose=1, patience=5)])

Epoch 1/5
100/100 [==============================] - 109s 1s/step - loss: 0.8860 - accuracy: 0.6553 - val_loss: 0.7079 - val_accuracy: 0.7500
Epoch 2/5
100/100 [==============================] - 100s 999ms/step - loss: 0.8893 - accuracy: 0.6441 - val_loss: 0.7834 - val_accuracy: 0.7396
Epoch 3/5
100/100 [==============================] - 102s 1s/step - loss: 0.8675 - accuracy: 0.6554 - val_loss: 0.7785 - val_accuracy: 0.7083
Epoch 4/5
100/100 [==============================] - 103s 1s/step - loss: 0.8483 - accuracy: 0.6626 - val_loss: 0.8801 - val_accuracy: 0.6458
Epoch 5/5
100/100 [==============================] - 107s 1s/step - loss: 0.8495 - accuracy: 0.6598 - val_loss: 0.7663 - val_accuracy: 0.7917


## Evaluate

In [4]:
model.evaluate(test_generator)

41/41 [==============================] - 32s 742ms/step - loss: 0.2139 - accuracy: 0.9270


[0.21387270092964172, 0.9270186424255371]

## Save

In [7]:
model.save('../models/efficientNetB3', save_format='tf')

2022-03-29 12:08:11.785970: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/efficientNetB3/assets


/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/matined/anaconda3/envs/x-ray/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
